# 移动

In [21]:
import pandas as pd
import numpy as np
import __dis
import time
import geopandas as gpd
from app import *

In [2]:
sit_cmcc=pd.read_csv(open('G:/1-规划/评估/竞对站点评估/2019年11月/1-基础数据/移动/输出数据/汇总/移动汇总.csv',encoding='gbk'))

In [3]:
gongcan = pd.read_csv(open('g:/1-规划/工参/工参数据/ltejizhan20191212.csv',encoding='gbk'))

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (15,17,22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
sit_cmcc = sit_cmcc.merge(gongcan[[ '小区中文名', '所属地区']].drop_duplicates('小区中文名'),how='left',on='小区中文名')

In [5]:
sit_cmcc.columns

Index(['小区中文名', '网络制式(TDD/FDD)', 'CellLongitude', 'CellLatitude', '覆盖类型',
       '区县归属', '移动业务域', '武汉行政域', '厂家名称', 'Indoor', 'FDD900', 'FDD1800', 'F',
       'D', 'E', 'A', '频段_900M', '频段_1800M', '频段_2100M', '所属地区'],
      dtype='object')

In [6]:
res_table = gongcan.groupby(by='所属地区').agg({'小区中文名':'count'})
res_table.rename(columns={'小区中文名':'小区总计'},inplace=True)

res_table = res_table.merge(sit_cmcc.groupby(by='所属地区').agg({'小区中文名':'count'}),left_index=True,right_index=True)
res_table.rename(columns={'小区中文名':'站址总计'},inplace=True)

In [7]:
list_pin = ['频段_900M','频段_1800M','频段_2100M']

In [8]:
for pin in list_pin:
    sit_cmcc_use = sit_cmcc[sit_cmcc[pin]==pin.split('_')[1]]
    res_table = res_table.merge(sit_cmcc_use.groupby(by='所属地区').agg({'小区中文名':'count'}),left_index=True,right_index=True)
    res_table.rename(columns={'小区中文名':pin},inplace=True)

In [9]:
res_table

,小区总计,站址总计,频段_900M,频段_1800M,频段_2100M
所属地区,,,,,
十堰,23619,4161,2561,3320,846
咸宁,16039,2759,1472,2289,541
天门,4903,893,413,795,158
孝感,21687,3363,1788,2959,611
宜昌,29658,5021,3040,4302,1027
恩施,21602,3985,2639,3086,634
武汉,85817,8278,5250,6811,4183
江汉,5526,927,410,824,217
潜江,4608,840,368,729,158


In [12]:
def table_sum(df):
    sum_column = df.sum(axis=1)
    sum_column.name='总计'
    sum_row = df.sum()
    sum_row.name='总计'
    df = pd.concat([df,sum_column],axis=1)
    df = df.append(sum_row)
    return df

In [13]:
table_sum(res_table)

,小区总计,站址总计,频段_900M,频段_1800M,频段_2100M,总计
所属地区,,,,,,
十堰,23619.0,4161.0,2561.0,3320.0,846.0,34507.0
咸宁,16039.0,2759.0,1472.0,2289.0,541.0,23100.0
天门,4903.0,893.0,413.0,795.0,158.0,7162.0
孝感,21687.0,3363.0,1788.0,2959.0,611.0,30408.0
宜昌,29658.0,5021.0,3040.0,4302.0,1027.0,43048.0
恩施,21602.0,3985.0,2639.0,3086.0,634.0,31946.0
武汉,85817.0,8278.0,5250.0,6811.0,4183.0,110339.0
江汉,5526.0,927.0,410.0,824.0,217.0,7904.0
潜江,4608.0,840.0,368.0,729.0,158.0,6703.0


# 联通电信

In [24]:
dianxin = pd.read_excel('g:/1-规划/评估/竞对站点评估/2019年11月/2-处理过程/电信整理汇总.xlsx',encoding = 'gbk',sheet_name='Sheet1')
liantong = pd.read_excel('g:/1-规划/评估/竞对站点评估/2019年11月/2-处理过程/联通汇总.xlsx',encoding = 'gbk',sheet_name='Sheet1')

In [25]:
dianxin_h = dianxin.loc[dianxin['室内外']=='室外']
liantong_h = liantong.loc[liantong['室内外']=='室外']

In [26]:
hubei = gpd.read_file('g:/1-规划/图层/湖北省地市-边界-边界/全省地市边界.TAB',encoding = 'gbk')
hubei = hubei.to_crs({'init':'epsg:4326'})

C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [27]:
dianxin_h = dianxin_h.astype(str)

dianxin_h.info()

dianxin_h['经度'] = dianxin_h['经度'].str.strip()
dianxin_h['纬度'] = dianxin_h['纬度'].str.strip()
dianxin_h['纬度'] = dianxin_h['纬度'].str.replace('?','')
dianxin_h['经度'] = pd.to_numeric(dianxin_h['经度'])
dianxin_h['纬度'] = pd.to_numeric(dianxin_h['纬度'])

dianxin['经度'] = dianxin['经度'].str.strip()
dianxin['纬度'] = dianxin['纬度'].str.strip()
# liantong['经度'] = liantong['经度'].str.strip()
# liantong['纬度'] = liantong['纬度'].str.strip()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112030 entries, 0 to 129758
Data columns (total 13 columns):
地市               112030 non-null object
小区名              112030 non-null object
基站名              112030 non-null object
经度               112030 non-null object
纬度               112030 non-null object
频点               112030 non-null object
频率               112030 non-null object
室内外              112030 non-null object
带宽               112030 non-null object
厂家               112030 non-null object
900-1800-2400    112030 non-null object
基站类型-室内外         112030 non-null object
微站-宏站-室分         112030 non-null object
dtypes: object(13)
memory usage: 12.0+ MB


In [28]:
dianxin_use = dianxin_h.loc[(dianxin_h['经度']>60) & (dianxin_h['纬度']<60)]
liantong_use = liantong_h.loc[(liantong_h['经度']>60 & (liantong_h['纬度']<60))]

dianxin_p = tools_geo.add_points(dianxin_use ,lon='经度',lat='纬度')
liantong_p = tools_geo.add_points(liantong_use , lon='经度',lat='纬度')

In [29]:
dianxin_res = gpd.sjoin(hubei,dianxin_p)
liantong_res = gpd.sjoin(hubei,liantong_p)

dianxin_res.drop(['geometry','index_right'],axis=1,inplace = True)
liantong_res.drop(['geometry','index_right'],axis=1,inplace = True)

In [30]:
dianxin_res['kkk'] = dianxin_res['kkk'].str.replace('市','')
liantong_res['kkk'] = liantong_res['kkk'].str.replace('市','')

dianxin_res_groupby = dianxin_res.groupby(by = '900-1800-2400')
liantong_res_groupby = liantong_res.groupby(by = '900-1800-2400')

In [31]:
dishi = dianxin_res.groupby(by = 'kkk').agg({'地市':'count'})
dishi.columns = ['电信小区']
liantong_linshi_1=liantong_res.groupby(by = 'kkk').agg({'地市':'count'})
liantong_linshi_1.columns = ['联通小区']
dishi['联通小区']=0
dishi['联通小区'].update(liantong_linshi_1['联通小区'])

In [32]:
dianxin_res_qc1 = dianxin_res.drop_duplicates('基站名')
dianxin_linshi_2 = dianxin_res_qc1.groupby(by = ['kkk']).agg({'kkk':'count'})
dianxin_linshi_2.columns = ['电信总站址']
dishi['电信总站址'] = 0
dishi['电信总站址'].update(dianxin_linshi_2['电信总站址'])

In [33]:
liantong__res_qc1 = liantong_res.drop_duplicates('基站名')
liantong_linshi_2 = liantong__res_qc1.groupby(by = ['kkk']).agg({'kkk':'count'})
liantong_linshi_2.columns = ['联通总站址']
dishi['联通总站址'] = 0
dishi['联通总站址'].update(liantong_linshi_2['联通总站址'])

In [34]:
for name,dianxin_t in dianxin_res_groupby:
    dianxin_res_qc = dianxin_t.drop_duplicates('基站名')
    dianxin_linshi_2 = dianxin_res_qc.groupby(by = ['kkk']).agg({'kkk':'count'})
    dianxin_linshi_2.columns = [name+'电信总站址']
    dishi[name+'电信总站址'] = 0
    dishi[name+'电信总站址'].update(dianxin_linshi_2[name+'电信总站址'])

for name,liantong_t in liantong_res_groupby:
    liantong_res_qc = liantong_t.drop_duplicates('基站名')
    liantong_linshi_2 = liantong_res_qc.groupby(by = ['kkk']).agg({'kkk':'count'})
    liantong_linshi_2.columns = [name+'联通总站址']
    dishi[name+'联通总站址'] = 0
    dishi[name+'联通总站址'].update(liantong_linshi_2[name+'联通总站址'])

In [35]:
a= dishi.sum()
a.name = '总计'
dishi =dishi.append(a)

In [37]:
dishi.to_clipboard()